There is a lot more that you can do with outputs (such as including interactive outputs)
with your book. For more information about this, see [the Jupyter Book documentation](https://jupyterbook.org)

# Software vulnerabilities investigation

In [42]:
from concurrent.futures import ThreadPoolExecutor
import httpx
import zipfile
import tempfile
from pathlib import Path
import orjson as ojson
from typing import Any

## Prepare data

In [41]:
CVE_ZIP_URL = "https://github.com/CVEProject/cvelistV5/releases/download/cve_2023-12-09_1500Z/2023-12-09_all_CVEs_at_midnight.zip.zip"
# Download archive
resp = httpx.get(CVE_ZIP_URL, follow_redirects=True)

with tempfile.TemporaryDirectory() as tmpdir:
    tmp_path = Path(tmpdir)

    with (tmp_path / "cve.zip.zip").open("wb") as f:
        f.write(resp.content)
   
    with zipfile.ZipFile(f.name, 'r') as zip_zip_ref:
        tmp_zip = tmp_path / "cves.zip"
        zip_zip_ref.extractall(tmp_path)
        with zipfile.ZipFile(tmp_zip) as zip_ref:
            files_to_extract = [name for name in zip_ref.namelist() if name.startswith(("cves/2022/", "cves/2023/"))]
            zip_ref.extractall(members=files_to_extract)


In [46]:
cves_data: list[dict[str, Any]] = []
cves_folder = Path("cves/")
with ThreadPoolExecutor(20) as p:
    for f in p.map(lambda fp: fp.read_text(), cves_folder.rglob("*.json")):
        cves_data.append(ojson.loads(f))
        
print(len(cves_data))

46226


## Process CWE types

In [48]:
# Get only cves_with cwe
cves_with_cwe_data: list[dict[str, Any]] = []
for cve_data in cves_data:
    problem_types = cve_data["containers"]["cna"].get("problemTypes")
    if problem_types:
        cves_with_cwe_data.append(cve_data)
        
print(len(cves_with_cwe_data))

43434
